# 02 — Признаки и целевая переменная

Загружаем M15 и H1 данные, считаем индикаторы на M15, формируем трендовый признак на H1,
объединяем признаки, строим 3-барную целевую доходность и сохраняем итоговый датасет.

In [12]:
from pathlib import Path
import sys
import pandas as pd

PROJECT_ROOT = Path.cwd().resolve()
if (PROJECT_ROOT / "src").exists():
    ROOT = PROJECT_ROOT
elif (PROJECT_ROOT.parent / "src").exists():
    ROOT = PROJECT_ROOT.parent
else:
    ROOT = PROJECT_ROOT

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.features import (
    build_m15_features,
    build_h1_trend_features,
    merge_m15_with_h1,
    add_target,
    drop_na_for_training,
)

DATA_DIR = ROOT / "data"
m15_path = DATA_DIR / "eurusd_M15.parquet"
h1_path = DATA_DIR / "eurusd_H1.parquet"
output_path = DATA_DIR / "eurusd_features.parquet"

In [6]:
df_m15 = pd.read_parquet(m15_path)
df_h1 = pd.read_parquet(h1_path)

df_m15["time"] = pd.to_datetime(df_m15["time"])
df_h1["time"] = pd.to_datetime(df_h1["time"])

df_m15 = df_m15.sort_values("time").reset_index(drop=True)
df_h1 = df_h1.sort_values("time").reset_index(drop=True)


In [7]:
df_m15_feat = build_m15_features(df_m15.copy())
df_m15_feat.head()


,time,open,high,low,close,volume,ema_20,ema_50,ema_20_50_diff,rsi_14,adx_14,atr_14,ret_1,ret_3
0,2025-10-08 12:15:00+00:00,1.16352,1.16368,1.16314,1.16354,1692,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
1,2025-10-08 12:30:00+00:00,1.16354,1.16380,1.16296,1.16351,1670,NaN,NaN,NaN,NaN,0.0,0.0,-0.000026,NaN
2,2025-10-08 12:45:00+00:00,1.16350,1.16358,1.16318,1.16324,1986,NaN,NaN,NaN,NaN,0.0,0.0,-0.000232,NaN
3,2025-10-08 13:00:00+00:00,1.16324,1.16399,1.16313,1.16370,2376,NaN,NaN,NaN,NaN,0.0,0.0,0.000395,0.000138
4,2025-10-08 13:15:00+00:00,1.16372,1.16448,1.16310,1.16311,2275,NaN,NaN,NaN,NaN,0.0,0.0,-0.000507,-0.000344


In [8]:
df_h1_trend = build_h1_trend_features(df_h1.copy())
df_h1_trend.head()


,time,ema_50_h1,h1_trend_flag
0,2025-03-04 08:00:00+00:00,NaN,0
1,2025-03-04 09:00:00+00:00,NaN,0
2,2025-03-04 10:00:00+00:00,NaN,0
3,2025-03-04 11:00:00+00:00,NaN,0
4,2025-03-04 12:00:00+00:00,NaN,0


In [9]:
df_merged = merge_m15_with_h1(df_m15_feat, df_h1_trend)
df_merged.head()


,time,open,high,low,close,volume,ema_20,ema_50,ema_20_50_diff,rsi_14,adx_14,atr_14,ret_1,ret_3,ema_50_h1,h1_trend_flag
0,2025-10-08 12:15:00+00:00,1.16352,1.16368,1.16314,1.16354,1692,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.166153,0
1,2025-10-08 12:30:00+00:00,1.16354,1.16380,1.16296,1.16351,1670,NaN,NaN,NaN,NaN,0.0,0.0,-0.000026,NaN,1.166153,0
2,2025-10-08 12:45:00+00:00,1.16350,1.16358,1.16318,1.16324,1986,NaN,NaN,NaN,NaN,0.0,0.0,-0.000232,NaN,1.166153,0
3,2025-10-08 13:00:00+00:00,1.16324,1.16399,1.16313,1.16370,2376,NaN,NaN,NaN,NaN,0.0,0.0,0.000395,0.000138,1.166037,0
4,2025-10-08 13:15:00+00:00,1.16372,1.16448,1.16310,1.16311,2275,NaN,NaN,NaN,NaN,0.0,0.0,-0.000507,-0.000344,1.166037,0


In [10]:
df_with_target = add_target(df_merged, horizon=3)
df_with_target[["time", "close", "target"]].head(10)


,time,close,target
0,2025-10-08 12:15:00+00:00,1.16354,0.000138
1,2025-10-08 12:30:00+00:00,1.16351,-0.000344
2,2025-10-08 12:45:00+00:00,1.16324,0.000266
3,2025-10-08 13:00:00+00:00,1.16370,-0.000447
4,2025-10-08 13:15:00+00:00,1.16311,0.000232
5,2025-10-08 13:30:00+00:00,1.16355,-0.000198
6,2025-10-08 13:45:00+00:00,1.16318,0.000069
7,2025-10-08 14:00:00+00:00,1.16338,-0.001049
8,2025-10-08 14:15:00+00:00,1.16332,-0.001496
9,2025-10-08 14:30:00+00:00,1.16326,-0.001315


In [11]:
df_final = drop_na_for_training(df_with_target)
print(df_final.shape)

df_final.to_parquet(output_path, index=False)
print(f"Saved final feature dataset to: {output_path}")


(4947, 17)
Saved final feature dataset to: /Users/oleksandrpc/Desktop/my projects/EUR_USD_MODEL/data/eurusd_features.parquet


Итоговый датасет `eurusd_features.parquet` будет использоваться в ноутбуке 03 для обучения модели
и проверки стратегии. Признаки и целевая переменная готовы к обучению.